In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Notebooks/Repository/go-emotions/notebooks

MessageError: ignored

In [4]:
%run library_utils.ipynb

import os 
import numpy as np 
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from torch import Tensor
import torch.nn as nn

tqdm.pandas()

module = 'go-emotion-glove'
args = load_args(module)

device: cuda


### Tokenizer & Encoder

## Loading dataset

In [5]:
dataset_path = os.path.join(args.output_dir, args.train_dataset_path)
dataset_path

df = pd.read_csv(dataset_path)
df.head(3)

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
0,"oh yah , no worries . i appreciate your time a...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"name just has the runs . don t worry , name i...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,"it s the donuts done brought ya back , is not...",0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from torchtext.vocab import GloVe, Vocab, vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import pandas as pd

In [134]:
class GloveVocab(Vocab):
  def __init__(self, 
               ordered_dict, 
               min_freq=1, 
               unk_token='<unk>', 
               pad_token='<pad>'):

    # min_freq     
    tokens = []
    for token, freq in ordered_dict.items():
      if freq >= min_freq:
        tokens.append(token)   
    super().__init__(VocabPybind(tokens, None))

    # add special tokens
    self.unk_token = unk_token
    self.pad_token = pad_token
    self.special_tok = {
        unk_token: len(self), 
        pad_token: len(self) + 1
    }
    self.insert_token(unk_token, self.special_tok[unk_token])
    self.set_default_index(self.special_tok[unk_token])
    self.insert_token(pad_token, self.special_tok[pad_token])

  def save_pretrained(self, path):
    if not os.path.exists(path):
      os.makedirs(path)
    torch.save(self, pathjoin(path, 'tokenizer.pt'))
    return 

  @classmethod 
  def from_pretrained(cls, path):
    return torch.load(pathjoin(path, 'tokenizer.pt'))


In [121]:
def load_glove(name, dim, cache='/contents/.vector_cache'):
  glove_vec = GloVe('twitter.27B', dim=200, cache='/contents/.vector_cache')
  glove_vocab = GloveVocab(glove_vec.stoi)
  glove_vec.vectors = torch.cat([glove_vec.vectors, glove_vec.vectors.sum(1).view(-1, 1)], axis=1)
  return glove_vec, glove_vocab

In [133]:
torch.cat([glove_vec.vectors, glove_vec.vectors.sum(1).view(-1, 1)], axis=1)

tensor([[ 3.1553e-01,  5.3765e-01,  1.0177e-01,  ..., -4.6540e-01,
          5.2066e-01, -1.6427e+00],
        [ 3.5132e-01,  5.6084e-04, -2.1488e-01,  ..., -3.7206e-02,
          8.5384e-01, -6.6361e-01],
        [ 8.0767e-01,  4.9786e-01,  8.2696e-02,  ..., -4.7785e-01,
          6.7059e-01, -4.4268e+00],
        ...,
        [ 4.6298e-01,  1.0477e-01,  2.3461e-01,  ..., -3.2021e-01,
         -5.3523e-01, -2.1072e+00],
        [ 5.9501e-01, -2.1580e-01,  2.9049e-01,  ...,  1.2880e-01,
         -3.8757e-02,  1.0202e+01],
        [-5.9487e-01, -1.9176e-01,  3.7470e-01,  ..., -1.9884e-01,
         -9.2338e-01, -3.7806e+00]])

In [135]:

glove_vec = GloVe('twitter.27B', dim=200, cache='/contents/.vector_cache')
glove_vocab = GloveVocab(glove_vec.stoi)

len(glove_vocab)
  


1193515

In [124]:
len(glove_vec.stoi), glove_vec.vectors.shape

(1193514, torch.Size([1193514, 200]))

In [ ]:
glove_vocab

In [113]:
glove_vec, glove_vocab = load_glove('twitter.27B', 200)

In [115]:
glove_vec.vectors.shape

torch.Size([1193514, 200])

In [116]:
len(glove_vocab)

1193515

In [20]:
def tokenize_glove_inputs(texts, vocab, maxlen=100):
  X_tk = []
  for text in texts:
    tokens = [vocab[word] for word in text.split(' ')[:maxlen]]
    tokens = Tensor(tokens).type(torch.int32)
    X_tk.append(tokens)

  pad_tok_val = vocab.special_tok[vocab.pad_token]
  X_tk = pad_sequence(X_tk, batch_first=True, padding_value=pad_tok_val)
  
  if X_tk.shape[-1] < maxlen:
    pad_len = maxlen - X_tk.shape[-1]
    pad = torch.full((X_tk.shape[0], pad_len), pad_tok_val)
    X_tk = torch.cat([X_tk, pad], axis=1)

  return {'input_ids': X_tk}


X_tk = tokenize_glove_inputs(df['text'][:10000], glove_vocab, maxlen=args.sentence_max_len)

In [21]:
def _generate_glove_dataset(
    X, 
    y, 
    vocab,
    sentence_max_len = 82,
    split=None
):
    if split is None:
        X_tk = tokenize_glove_inputs(
            X.tolist(), 
            vocab=vocab,
            maxlen=sentence_max_len
        )
        dataset = ZipDataset({
            'input_ids': X_tk['input_ids'], 
            'y_true': Tensor(y).type(torch.float32)
        })
        return dataset
    else:
        X1, X2, y1, y2 = train_test_split(
            X, y, 
            test_size=split, 
            shuffle=False,
        )
        d1 = _generate_glove_dataset(X1, y1, vocab, sentence_max_len, split=None)
        d2 = _generate_glove_dataset(X2, y2, vocab, sentence_max_len, split=None)
        return (d1, d2)

def generate_glove_dataset(
    dataset_path, 
    vocab, 
    emotions, 
    sentence_max_len=82,
    split= None
):
    D = pd.read_csv(dataset_path)
    X = D['text'].to_numpy()
    y = D[list(emotions)].to_numpy()

    return _generate_glove_dataset(
        X, y, 
        vocab, 
        sentence_max_len, 
        split
    )

In [14]:
train_dataset_path = os.path.join(args.output_dir, args.train_dataset_path)
train_dataset, val_dataset = generate_glove_dataset(
    dataset_path = train_dataset_path, 
    vocab = glove_vocab, 
    emotions=args.emotions, 
    sentence_max_len = args.sentence_max_len, 
    split=args.validation_split_from_train
)

In [65]:
class GoEmotionGloVeGRUClassifer(nn.Module):
  def __init__(self,  
               n_vocab, 
               embedding_dim, 
               criterion=nn.BCEWithLogitsLoss(), 
               seq_len=82, 
               rnn_hidden=50, 
               rnn_num_layers=1, 
               bidirectional=True, 
               hiddens=None, 
               dropout_p=0.1, 
               n_cls = 28, 
               freeze_embedding = False
               ):
    self.config = {
        'freeze_embedding': freeze_embedding,
        'embedding_dim': embedding_dim,
        'n_vocab': n_vocab,
        'seq_len': seq_len, 
        'rnn_hidden': rnn_hidden, 
        'rnn_num_layers': rnn_num_layers, 
        'bidirectional': bidirectional, 
        'hiddens':  [50] if hiddens is None else hiddens, 
        'dropout_p': dropout_p, 
        'n_cls': n_cls
    }
    super().__init__()
    self.gru = nn.GRU(
        input_size = embedding_dim, 
        hidden_size = rnn_hidden, 
        batch_first = True, 
        bidirectional = bidirectional
    )
    self.dropout = nn.Dropout(dropout_p)
    self.criterion = criterion
    # full connected
    fcs = []
    in_feature = (int(bidirectional) + 1) * rnn_hidden
    for h in self.config['hiddens']:
      fcs.append(nn.Linear(in_feature, h))
      fcs.append(nn.ReLU())
    
    # final layer 
    fcs.append(nn.Linear(in_feature, n_cls))
    self.fcs = nn.Sequential(*fcs)
    self.embedding = None if freeze_embedding else nn.Embedding(n_vocab, embedding_dim)

  def forward(self, input_ids, y_true=None):
    emb = self.embedding(input_ids)
    output, _ = self.gru(emb)
    output = output[:, -1, :]
    z = self.dropout(output) 
    logits = self.fcs(z)

    if not (y_true is None):
      loss = self.criterion(logits, y_true)
      return (loss, logits)

    return logits

  def set_embedding(self, embedding):
    if embedding is not None and self.embedding is not None:
      assert embedding.weight.shape == self.embedding.weight.shape
    self.embedding = embedding 
    return 
  
  def set_embedding_from_vec(self, vec, freeze=True):
    embedding = nn.Embedding.from_pretrained(vec.vectors, freeze=freeze)
    self.set_embedding(embedding)
    return

  def set_freeze_embedding(self, do_freeze):
    self.config['freeze_embedding'] = do_freeze
    if self.embedding:
      self.embedding.requires_grad = False 
    return 
    
  def save_pretrained(self, path):
    if self.config['freeze_embedding']:
      embedding = self.embedding
      self.set_embedding(None)
    torch.save({
        'model': self.state_dict(), 
        'config': {
            'module': self.config, 
            'criterion': self.criterion.__class__.__name__,
            'architecture': str(self), 
        }
    }, pathjoin(path, 'model.pt'))
    if self.config['freeze_embedding']:
      self.set_embedding(embedding)
    return 

  @classmethod
  def from_pretrained(cls, path):
    pt_path = pathjoin(path, 'model.pt')
    checkpoint = torch.load(pt_path, map_location=torch.device('cpu'))
    config = checkpoint['config']
    model = cls(
          criterion = getattr(torch.nn, config['criterion'])(), 
          **config['module']
    )
    model.load_state_dict(checkpoint['model'])
    return model

In [66]:
def create_go_emotion_glove_gru(args, glove_vec):
  embedding = nn.Embedding.from_pretrained(glove_vec.vectors, freeze=True)
  (n_vocab, embedding_dim) = embedding.weight.shape
  model = GoEmotionGloVeGRUClassifer(
      n_vocab, 
      embedding_dim, 
      seq_len=args.sentence_max_len,
      rnn_hidden=args.rnn_hidden, 
      rnn_num_layers = args.rnn_num_layers, 
      bidirectional=args.bidirectional, 
      hiddens = args.fc_hiddens, 
      dropout_p = args.dropout_p, 
      n_cls = len(args.emotions), 
      freeze_embedding = True
  )
  model.set_embedding(embedding)

  return model 

In [67]:
model = create_go_emotion_glove_gru(args, glove_vec)

In [68]:
args.model_name = 'go-emotion-glove-gru'

In [69]:
save_checkpoint(
    model, 
    archive_dir=args.output_dir, 
    model_name = args.model_name, 
    checkpoint_id='null-model', 
    metadata = dict(args), 
    tokenizer = glove_vocab
)

'../outputs/go-emotion-glove-gru'

In [70]:
model_copy, vocab_copy, metadata = load_from_checkpoint(
    args.output_dir, 
    model_name = args.model_name, 
    load_tokenizer = True, 
    model_cls = GoEmotionGloVeGRUClassifer, 
    tok_cls = GloveVocab
)
model_copy.set_embedding_from_vec(glove_vec, freeze=True)

## Utility to predict on examples

In [79]:
def proba_on_examples(texts, model):
    X_tk = tokenize_glove_inputs(
        texts, glove_vocab, maxlen = args.sentence_max_len
    )
    proba = predict_proba_examples(X_tk, model)
    return proba

In [91]:
X_tk['input_ids'].shape

torch.Size([10000, 62])

In [88]:
model.embedding

Embedding(1193514, 200)

In [87]:
X_tk

{'input_ids': tensor([[    192,     818,       3,  ..., 1193514, 1193514, 1193514],
         [    589,      58,     214,  ..., 1193514, 1193514, 1193514],
         [     32, 1193513,     136,  ..., 1193514, 1193514, 1193514],
         ...,
         [      9,     224,    1664,  ..., 1193514, 1193514, 1193514],
         [1193513,      79,    1816,  ..., 1193514, 1193514, 1193514],
         [    177,     210,      15,  ..., 1193514, 1193514, 1193514]])}

In [96]:
X_tk['input_ids'].view(-1)

tensor([    192,     818,       3,  ..., 1193514, 1193514, 1193514])

In [108]:
glove_vocab.lookup_token(1193514)

'<pad>'

In [106]:
glove_vocab['<pad>']

1193514

In [112]:
glove_vec.vectors.shape

torch.Size([1193514, 200])

In [109]:
model.embedding(torch.Tensor([3, 3, 2, 1, 5, 1193514]).type(torch.int32))

IndexError: ignored

In [80]:
proba = proba_on_examples(['hello my name is jeongwon', 'nice to meet you!'], model)
proba_to_emotion(
    proba,
    args.classification_threshold, 
    args.emotions, 
)[0][:3]

IndexError: ignored

In [ ]:
proba = predict_proba_examples(val_dataset[:5], model)
y_true = val_dataset[:5]['y_true'].numpy()
compute_classification_metrics(y_true, proba, 0.1)

In [ ]:
# testing evaluate
# evaluate(
#     model, 
#     val_dataset, 
#     batch_size = args.eval_batch_size, 
#     threshold = args.classification_threshold, 
#     device = args.device
# )

# testing saving
# model = create_go_emotion_pool(args, encoder)
# _ = save_checkpoint(
#     model, 
#     args.output_dir, 
#     model_name='testmodel',
#     metadata=args
# )

# model_copy, _ = load_from_checkpoint(
#     args.output_dir,
#     model_name='testmodel',
#     checkpoint_id='null-model', 
#     model_cls=GoEmotionPoolClassifer
# )

# check_model_same(model, model_copy)

In [ ]:
# clear_archive(args.output_dir, args.model_name)

In [ ]:
model = create_go_emotion_pool(args, encoder)
model, metadata = train(
    model, 
    train_dataset, 
    val_dataset, 
    dict(args), 
    tokenizer, 
    epochs=args.train_epochs, 
    train_batch_size = args.train_batch_size, 
    val_batch_size = args.eval_batch_size, 
    save_steps = args.save_steps, 
    validation_steps= args.validation_steps, 
    archive_dir = args.output_dir, 
    model_name = args.model_name, 
    classification_threshold = args.classification_threshold, 
    learning_rate = args.learning_rate, 
    grad_clip_max = args.grad_clip_max, 
    weight_decay = args.weight_decay, 
    warmup_ratio = args.warmup_ratio, 
    logging_metrics= ['macro_f1', 'macro_precision', 'macro_recall'],
    continue_training=False,
    device = args.device
)

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

df = get_attrs_from_checkpoints_meta(args.output_dir, args.model_name, [
    'tr_metrics.loss', 
    'tr_metrics.trigger_rate',
    'val_metrics.loss', 
    'val_metrics.trigger_rate', 
    'val_metrics.accuracy', 
    'val_metrics.macro_f1',
    'val_metrics.micro_f1'
])
df.head(3)

In [ ]:
plt.plot(df['checkpoint_id'][3:], df['tr_metrics.loss'][3:], label='train')
plt.plot(df['checkpoint_id'][3:], df['val_metrics.loss'][3:], label='val')
plt.legend()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(df['checkpoint_id'], 
         df['val_metrics.macro_f1'], 
         label='val-f1', c='blue')
ax2.plot(df['checkpoint_id'], 
         df['val_metrics.trigger_rate'], 
         label='val-trigger_rate', c='red')

ax1.legend()
ax2.legend()

## Checking final model 


In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

model, meta = load_from_checkpoint(
    args.output_dir, 
    args.model_name, 
    checkpoint_id='120000', 
    load_metadata=True,
    load_model=True, 
    model_cls=GoEmotionPoolClassifer
)

proba = predict_proba(
    model, 
    val_dataset, 
    batch_size = args.eval_batch_size, 
    device = args.device
)
y_true = val_dataset[:]['y_true'].numpy()

val_final = compute_classification_metrics(y_true, proba, threshold=args.classification_threshold)
print(val_final)

for i, emt in enumerate(args.emotions):
    fpr, tpr, _ = roc_curve(y_true[:, i], proba[:, i])
    plt.plot(fpr, tpr)
    plt.title(emt)
    plt.show()


In [ ]:
proba_to_emotion(
    proba_on_examples([
        'hi nice to meet you', 
        'yes!! I got accepted', 
        'god dam it why would you do that?', 
        'i am terribly sorry',
        'i love you',
        'what is wrong with the weather today?'
    ], model), 
    0.4, 
    args.emotions)